In [ ]:
import pandas as pd
import yfinance as yf
from datetime import date
import sys
import streamlit as st
import plotly.express as px
import xml.etree.ElementTree as ET

In [ ]:
#Define caminho arquivo B3
caminho = "C:\\Users\\adona\\Downloads\\lista_B3"
arquivo_b3 = "BVBG.028.02_BV000327202601260327141936752293290.xml"
caminho_arquivo_b3 = f'{caminho}\{arquivo_b3}'

In [ ]:
# Le o arquivo xml 
tree = ET.parse(caminho_arquivo_b3)
root = tree.getroot()

NS = {
    "d": "urn:bvmf.100.02.xsd"
}

instrms = root.findall(".//d:Instrm", NS)

In [ ]:
#traz todos os tickers em uma variável

tckrs = []

for instrm in instrms:
    t = instrm.findtext(".//d:TckrSymb", default=None, namespaces=NS)
    if t:
        tckrs.append(t.strip())

In [ ]:
#Filtra tickers no padrão spot (4 letras + 1 ou 2 números)

tickers = []

for t in tckrs:
    t = t.upper()

    if len(t) not in (5, 6):
        continue

    if not t[:4].isalpha():
        continue

    if not t[4:].isdigit():
        continue

    tickers.append(t)

In [ ]:
#Traz tickers ação

tickers_acao = []

for ticker in tickers:
    
    ticker_num = ticker[4:]

    if ticker_num not in ('3', '4'):
        continue

    tickers_acao.append(ticker)

In [ ]:
#Traz tickers BDR

tickers_bdr = []

for ticker_bdr in tickers:

    if len(ticker_bdr) > 6:
        continue

    if not ticker_bdr[4:] == "34":
        continue

    tickers_bdr.append(ticker_bdr)

In [ ]:
#Traz tickers FII

tickers_FII = []

for instrm_fii in instrms:
    t = instrm_fii.findtext(".//d:TckrSymb", default="", namespaces=NS).strip().upper()
    u = instrm_fii.findtext("d:FinInstrmAttrCmon/d:Desc", default="", namespaces=NS).upper()
    v = instrm_fii.findtext("d:InstrmInf/d:EqtyInf/d:SpcfctnCd", default="", namespaces=NS).upper()

    if (("FII" in u) or ("IMOBI" in u)) and ("CI" in v) and (len(t) < 7) and t[4:] == "11":
        tickers_FII.append(t.strip())

In [ ]:
#traz tickers UNIT

tickers_unit = []

for instrms_unit in instrms:
    t = instrms_unit.findtext(".//d:TckrSymb", default="", namespaces=NS).strip().upper()
    u = instrms_unit.findtext("d:FinInstrmAttrCmon/d:Desc", default="", namespaces=NS).upper()
    v = instrms_unit.findtext("d:InstrmInf/d:EqtyInf/d:SpcfctnCd", default="", namespaces=NS).upper()
    
    if ("UNT" in u) and ("UNT" in v) and (len(t) < 7) and t[4:] == "11":
        tickers_unit.append(t.strip())

    

In [ ]:
tickers_etf = []

for instrms_etf in instrms:
    t = instrms_etf.findtext(".//d:TckrSymb", default="", namespaces=NS).strip().upper()
    u = instrms_etf.findtext("d:FinInstrmAttrCmon/d:Desc", default="", namespaces=NS).upper()
    v = instrms_etf.findtext("d:InstrmInf/d:EqtyInf/d:SpcfctnCd", default="", namespaces=NS).upper()
    w = instrms_etf.findtext("d:InstrmInf/d:EqtyInf/d:CrpnNm", default="", namespaces=NS).upper()

    if ("CI" in u or "IVB" in u) and ("FII" not in u) and ("IMOBI" not in u) and ("CI" in v or "IVB" in v) and ("IND" in w or "ÍND" in w) and (len(t) < 7) and t[4:] == "11":
        tickers_etf.append(t.strip())